In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/standings?season=45&conference=11&division=-1&standingstype=division&context=overall&specialteams=false&sortkey=points&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for row in rows[1:]:  # Skip header row
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Team', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PCT', 'PCT2', 'PIM', 'RW', 'ROW', 'STK', 'IN-DIV', 'GPCT', 'P10', 'Nickname']


data_without_first_two = [row[2:] for row in acha_dynamic_stats]

df = pd.DataFrame(data_without_first_two, columns=column_names)


int_columns = ['RW', 'ROW', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PIM']
for column in int_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(int)


float_columns = ['PCT', 'GPCT', 'PCT2']
for column in float_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')


string_columns = ['Team', 'STK', 'IN-DIV', 'P10', 'Nickname']
for column in string_columns:
    df[column] = df[column].astype(str)


df = df[df['Team'] != 'None']
df = df.dropna(subset=['Team'])

print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_wd1_team.csv', index=False)

     Team  GP   W   L  OTL  T  SOW  SOL  PTS  OTW  ...    PCT  PCT2  PIM  RW  \
0      AC  22  17   3    0  2    0    0   36    3  ...  0.818   0.0  223  14   
1    ITW1  18  13   3    1  1    0    0   28    3  ...  0.778   0.0  146  10   
2   MichS  12   9   2    1  0    0    0   19    1  ...  0.792   0.0   97   8   
3    Mich  14   8   3    0  3    0    0   19    1  ...  0.679   0.0   80   7   
4   MichD  17   8   7    2  0    0    0   18    0  ...  0.529   0.0  141   8   
5   Miami  15   7   5    2  1    0    0   17    0  ...  0.567   0.0  124   7   
6    LSSU  16   8   7    1  0    0    0   17    2  ...  0.531   0.0  116   6   
7     PSU  10   5   4    1  0    0    0   11    0  ...  0.550   0.0   68   5   
8    CUAA  12   3   9    0  0    0    0    6    0  ...  0.250   0.0  121   3   
9    GVSU  15   1  13    1  0    0    0    3    0  ...  0.100   0.0  138   1   
10     DU  12   0  11    0  1    0    0    1    0  ...  0.042   0.0  109   0   
11     AQ   8   0   8    0  0    0    0 